In [1]:
import re
import os

In [2]:
def readChordProFile(chordProFile):
    with open(chordProFile, 'r') as file:
        print(chordProFile)
        content = file.read()
        return content

In [ ]:
uniqueChordNames = ["A","B", "C", "D", "E", "F", "G"]    

In [ ]:
def cleanChords(chords):
    finalChords = []
    for chord in chords:
        chord = chord.replace("H", "B")
        chord = chord.replace("b", "-")
        chord = chord.replace("min", "m")
        chord = chord.replace("maj", "")
        chord = chord.replace("sus", "")
        chord = chord.replace("dim", "")
        if len(chord) > 3:
            print('Curring long chord ', chord)
            chord = chord[0:3]
        chord = chord.replace("7", "")        
        chord = chord.replace("(", "")
        chord = chord.replace("+", "")
        chord = chord.replace("?", "")
        chord = chord.split("/")[0]
        chord = chord.split("+")[0]
        chord = re.sub(r'\d+', '', chord)
        
        if chord[0:1] in uniqueChordNames:
            finalChords.append(chord)
    return finalChords

In [3]:
class Song:
    def __init__(self, chordProFile):
        self.chordProFile = chordProFile
        self.content = readChordProFile(chordProFile)
        self.chorus = self.getChorus()
        self.chords = self.getChords()
        self.title = self.getTitle()
        self.key = self.getKey()
        
    def getChords(self):
        chords = []
        
        for line in self.content.split("\n"): 
            chord_pattern = r'\[([^\]]+)\]'
            lineChords = []
            if "chorus}" in line.lower():
                lineChords = self.getChorus()
            elif "[" in line:
                lineChords = re.findall(chord_pattern, line)   
            chords.extend(cleanChords(lineChords))
        print('Chord of the song: ' + self.chordProFile, chords)
        return chords
        
    def getChorus(self):
        pattern = r"{soc}(.*?){eoc}"
        match = re.search(pattern, self.content, re.DOTALL)
        chords = []
        if match:
            chorus = match.group(1).strip()
            chord_pattern = r"\[([^\]]*)\]"
            chords = re.findall(chord_pattern, chorus)
            print("Chords in chorus:", chords)
        else:
            print("Chorus section not found.")
        return chords
    
    def getTitle(self):
        pattern = r"\{title:\s*(.*?)\s*\}"
        match = re.search(pattern, self.content)
        title = ''
        if match:
            title = match.group(1)
            print("The title is:", title)
        else:
            print("Title information not found, using the file name.")
            title = os.path.splitext(os.path.basename(self.chordProFile))[0]
            print("The file name is being used as title:", title)
        return title
    
    def getKey(self):
        pattern = r"{key:\s*([A-G][#b]?)\s*}"
        match = re.search(pattern, self.content)
        key = ''
        if match:
            key = match.group(1)
            print("The key is:", key)
        else:
            print("Key information not found.")
        return key

In [4]:
def createChordDictFromChordProFiles(dataFolderPath):
    fileNames = os.listdir(dataFolderPath)
    chordsDict = {}
    longestChordLength = 0
    for file in fileNames:
        print(file)
        if '.DS_Store' in file:
            continue
        chordproFile = dataFolderPath + file
        song = Song(chordproFile)
        chordsDict[song.title] = song.chords
    return chordsDict

In [ ]:
pathOfChordTxtFiles = './data/chordpro/'

songDict = createChordDictFromChordProFiles(pathOfChordTxtFiles)
print(songDict)